<a href="https://colab.research.google.com/github/osnow/AR_mutant_response/blob/master/AR_mut_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.model_selection import KFold, LeaveOneOut, train_test_split
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, roc_curve, auc, roc_auc_score
from imblearn.over_sampling import BorderlineSMOTE
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

#Import Dataset

In [ ]:
fingerprints = pd.read_csv('path_to_data/Morg_FPs.csv',index_col=0)
#fingerprints = fingerprints.loc[:, fingerprints.var() != 0.0] #remove zero variance columns

zscales = pd.read_csv('path_to_data/zscale.csv',header=None)
zscales = zscales.iloc[:,1:]
#zscales = zscales.loc[:, zscales.var() != 0.0] #remove zero variance columns

classes = pd.read_csv('path_to_data/all_muts_classes_updated.csv', index_col=0)
zscales.index = classes.name

merged = fingerprints.merge(zscales,left_index=True,right_index=True)
print(merged.shape)


In [ ]:
merged.values[:5,2048:].shape

In [ ]:

ODM_data = pd.read_csv('path_to_data/ODM_mutants_train.csv',index_col=0)
ODM_classes = ODM_data.pop('class')

ODM_data.columns = merged.columns
print(ODM_data.shape)

X = pd.concat([merged, ODM_data])
print(X.shape)

y = pd.concat([classes['class'], ODM_classes])
print(y.shape)
dummy_y = np_utils.to_categorical(y)

# Make ODM mutant validation set

In [ ]:
# create odm mutant zscales from the bica mutants
odm_zscales = zscales.iloc[:45,:].drop('wt_bic')
# change index name 
odm_zscales.index = odm_zscales.index.str.split('_').str[0]+'_odm'
odm_zscales.shape

In [ ]:
#creat matrix of only odm fingerprints 
odm_fp = fingerprints.loc['d880e_odm',:].values
fps = pd.DataFrame([odm_fp]*44)
# combine fps and zscales
odm_new_muts = np.hstack((fps,odm_zscales))
# convert to DF with names as index
odm_new_muts = pd.DataFrame(data=odm_new_muts,index=odm_zscales.index) 
odm_new_muts['class'] = odm_truth['Ground truth'].values # add classes column

In [ ]:
odm_new_muts.to_csv('path_to_data/ODM_mutants_train.csv')

#Build NN model

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(neurons1=128, neurons2=32, epochs=100,dropout1=0.1,dropout2=0.1,mbs=16):
    # create model
    model = Sequential()
    model.add(Dense(neurons1, input_dim=X.shape[1], kernel_initializer='he_normal', activation='relu'))
    model.add(Dropout(dropout1))
    model.add(Dense(neurons2, kernel_initializer='he_normal', activation='relu'))
    model.add(Dropout(dropout2))
    model.add(Dense(4, kernel_initializer='he_normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# Column Transformer and over sample 

In [ ]:
from pickle import dump
scaler = StandardScaler()
scaler.fit(X_train_res[:,2048:])
dump(scaler, open('path_to_data/scaler.pkl','wb'))

In [ ]:
from sklearn.compose import make_column_transformer 

ct = make_column_transformer(
        (StandardScaler(), slice(2048,4148)) # make sure this shape matches zscales
    ,remainder='drop',verbose=True)

BSmote = BorderlineSMOTE()

# Run Random CV pipeline

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# create model
estimators = []
estimators.append(('standardize', ct))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, verbose=0)))
pipeline = Pipeline(estimators)
# define the grid search parameters
neurons1 = [256, 128, 64, 32]
neurons2 = [32, 16, 8]
epochs = [10, 20, 50, 100, 150]
mbs = [8, 16, 32, 64]
dropout1 = [0.0, 0.01, 0.1, 0.2, 0.3]
dropout2 = [0.0, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(mlp__neurons1=neurons1, mlp__neurons2=neurons2,mlp__epochs=epochs,mlp__dropout1=dropout1,mlp__dropout2=dropout2,mlp__mbs=mbs,mlp__class_weight=[class_weight,'auto'])
search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_grid, n_jobs=1, cv=5, n_iter=25)
search_result = search.fit(X, dummy_y)
# summarize results
print("Best: %f using %s" % (search_result.best_score_, search_result.best_params_))
means = search_result.cv_results_['mean_test_score']
stds = search_result.cv_results_['std_test_score']
params = search_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# CV pipeline with plotted curves

In [ ]:
ls_mb_size = [8, 16, 32, 64]
ls_epoch = [10, 20, 50, 100, 150]
ls_rate = [0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.08, 0.1, 0.2, 0.3]
ls_h_dim1 = [256, 128, 64, 32]
ls_h_dim2 = [64, 32, 16, 8]
acts = ['relu', 'elu']

max_iter = 25
save_results_to = './val_curves/'

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for iters in range(max_iter):
    
    mbs = np.random.choice(ls_mb_size)
    hdm1 = np.random.choice(ls_h_dim1)
    hdm2 = np.random.choice(ls_h_dim2)
    epoch = np.random.choice(ls_epoch)
    rate1 = np.random.choice(ls_rate)
    rate2 = np.random.choice(ls_rate) 
    act = np.random.choice(acts)

    k = 0

    for tr_ind, ts_ind in kf.split(X.values):
        k = k + 1

        ct.fit(X.values[tr_ind,:])
        X_train_trans = ct.transform(X.values[tr_ind,:])
        X_test_trans = ct.transform(X.values[ts_ind,:])
        X_train = np.hstack((X.values[tr_ind,:210],X_train_trans))
        X_test = np.hstack((X.values[ts_ind,:210],X_test_trans))
        
        X_train, y_train = BorderlineSMOTE().fit_resample(X_train, dummy_y[tr_ind,:])

        model = Sequential()
        model.add(Dense(hdm1, input_dim=X_train.shape[1], kernel_initializer='he_normal', activation=act))
        model.add(Dropout(rate1))
        model.add(Dense(hdm2, kernel_initializer='he_normal', activation=act))
        model.add(Dropout(rate2))
        model.add(Dense(4, kernel_initializer='he_normal', activation='softmax'))
        #model.summary()
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        history = model.fit(X_train, y_train, epochs=epoch, batch_size=mbs,  verbose=0, validation_data=(X_test,dummy_y[ts_ind,:]))
        
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        title2 = 'Loss Train iter = {}, fold = {}, mb_size = {},  epoch = {}, rate = ({},{}), dim = ({},{}), act = {}'.\
                          format(iters, k, mbs, epoch, rate1, rate2, hdm1, hdm2, act)
        plt.title(title2)
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.savefig(save_results_to + title2 + '.png', dpi = 150, bbox_inches = 'tight')
        plt.close()
    K.clear_session()

#Retrain with best params and predict

In [ ]:
X_train = X.loc[~X.index.str.contains('odm'),:]
X_test = X.loc[X.index.str.contains('odm'),:]
y_train = dummy_y[~X.index.str.contains('odm'),:]
y_test = dummy_y[X.index.str.contains('odm'),:]


In [ ]:
# Train on whole dataset (no split)
X_train, y_train = X, dummy_y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,dummy_y, test_size=0.2, random_state=42)


In [ ]:
X_train_res, y_train_res = BorderlineSMOTE().fit_resample(X_train, y_train)
#X_train_res, y_train_res = X_train, y_train

In [ ]:
X_train_trans = ct.fit_transform(X_train_res)
#X_test_trans = ct.transform(X_test)
X_train = np.hstack((X_train_res[:,:2048],X_train_trans))
#X_test = np.hstack((X_test.iloc[:,:210],X_test_trans))

In [ ]:
# model callbacks 
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='auto')
mc = ModelCheckpoint(filepath='path_to_model/best_model_new.h5', monitor='val_accuracy', mode='auto', verbose=1, save_best_only=True)

In [ ]:
# create and fit on train data using best hyperparameters 
model = create_model(neurons1=128,neurons2=32,epochs=50, dropout1=0.01,dropout2=0.01, mbs=32)
model.fit(X_train,y_train_res,verbose=1,epochs=100,batch_size=16,validation_split=0.2, callbacks=[early_stopping,mc])

In [ ]:
# RF baseline Train and test 
random_state = np.random.RandomState(0)
model = RandomForestClassifier(n_estimators=50,max_depth=10, random_state=random_state,verbose=0,n_jobs=-1)
model.fit(X_train,y_train_res)

In [ ]:
# SVM baseline Train and Test
model = OneVsRestClassifier(svm.SVC(kernel='rbf', C= 100, gamma = .001, probability=True,
                                 random_state=random_state))
model.fit(X_train,y_train_res)

# ODM predictions

In [ ]:
# compare preds to ground truth
odm_truth = pd.read_csv('path_to_results/ODM_predictions.csv')
y_true = odm_truth['Ground truth']
y_pred = odm_truth['Predicted']

report = classification_report(y_true, y_pred, output_dict=True,zero_division=0)
print("Test Report", report)
cr = pd.DataFrame(report).transpose()
cr

In [ ]:
# predict on ODM external validation set 
saved_model = load_model('best_model.h5')
odm_pred = saved_model.predict(X_test)
pred_classes = np.argmax(odm_pred, axis=1)
odm_pred_classes = pd.DataFrame(pred_classes,index=odm_zscales.index)
#odm_pred_classes.to_csv('/content/drive/My Drive/2019-McGill-Mutants/ODM_predictions.csv')



# Evaluate model and produce results

In [ ]:
# Train and test accuracy 
model = load_model('best_model.h5')
_, train_acc = model.evaluate(X_train, y_train_res, verbose=0)
_, test_acc = model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
# get results and report
y_pred_train = model.predict_proba(X_train)
y_pred_test = model.predict_proba(X_test)

# report accuracy
y_pred = np.argmax(y_pred_test, axis=1)
y_true = np.argmax(y_test, axis=1)
report = classification_report(y_true, y_pred, output_dict=True)
print("Test Report", report)
cr = pd.DataFrame(report).transpose()
cr.to_csv('DNN_class_report.csv')
cr

#Plot confusion matrix

In [ ]:
from mlxtend.evaluate import confusion_matrix
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix

cm = confusion_matrix(y_target=y_true, 
                      y_predicted=y_pred, 
                      binary=False)


fig, ax = plot_confusion_matrix(conf_mat=cm)
ax.set_xticklabels(['','non-resp.','antag.', 'agonist', 'mixed'])
ax.set_yticklabels(['','non-resp.','antag.', 'agonist', 'mixed'])
plt.savefig('SVM_test_CM.eps',dpi=300)
plt.show()

# ROC curve with baselines

In [ ]:

fig = plt.figure(figsize=(15,8))
random_state = np.random.RandomState(0)

kfold = KFold(n_splits=5, random_state=random_state, shuffle=True)


# Define baselines and NN 
svm_clf = OneVsRestClassifier(svm.SVC(kernel='rbf', C= 100, gamma = .001, probability=True,
                                 random_state=random_state))
rf_clf =  OneVsRestClassifier(RandomForestClassifier(n_estimators=50,max_depth=10, random_state=random_state,verbose=0,n_jobs=-1))
NN_clf = KerasClassifier(build_fn=create_model)

# DNN model callbacks 
early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='auto')
mc = ModelCheckpoint(filepath='./best_model.h5', monitor='val_acc', mode='auto', verbose=0, save_best_only=True)

names = ['SVM', 'RF', 'DNN']
classifiers = [svm_clf, rf_clf, NN_clf]

for i, classifier in enumerate(classifiers):
    tprs = []
    aucs = []

    fold = 0
    for train, test in kfold.split(X, dummy_y):
        X_train_res, y_train_res = BorderlineSMOTE().fit_resample(X.iloc[train,:], dummy_y[train,:])
        
        X_train_trans = ct.fit_transform(X_train_res)
        X_test_trans = ct.transform(X.iloc[test,:])
        X_train = np.hstack((X_train_res[:,:210],X_train_trans))
        X_test = np.hstack((X.iloc[test,:210],X_test_trans))

        if classifier == NN_clf:
            classifier.fit(X_train, y_train_res,callbacks=[early_stopping,mc],epochs=100,validation_split=0.2)
            #classifier = load_model('best_model.h5')
            probas_ = classifier.predict_proba(X_test)
        elif classifier == rf_clf:
            probas_ = classifier.fit(X_train, y_train_res).predict_proba(X_test)
        elif classifier == svm_clf:
            probas_ = classifier.fit(X_train, y_train_res).decision_function(X_test)   
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        #for i in range(4):
            #fpr[i], tpr[i], _ = roc_curve(y_train_res[test, i], probas_[:, i])
            #roc_auc[i] = roc_auc_score(y_train_res[test, i], probas_[:, i],average='weighted')

        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(dummy_y[test].ravel(), probas_.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        roc_auc['micro'] = roc_auc_score(dummy_y[test], probas_,average='weighted')
        
        mean_fpr = np.linspace(0, 1, 100)

        # append to list and plot for each fold
        tprs.append(interp(mean_fpr, fpr['micro'], tpr['micro']))
        tprs[-1][0] = 0.0
        aucs.append(roc_auc["micro"])
        #plt.plot(fpr['micro'], tpr['micro'], lw=2, alpha=0.3,
         #       label='ROC fold %d (AUC = %0.2f)' % (fold+1, roc_auc["micro"]))
        
        fold += 1

    mean_tpr_cv = np.mean(tprs, axis=0)
    mean_tpr_cv[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr_cv)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr_cv,
            label=r'%s ROC (AUC = %0.2f $\pm$ %0.2f)' % (names[i], mean_auc, std_auc),
            lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr_cv + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr_cv - std_tpr, 0)
    plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2)
                    #,label=r'$\pm$ 1 std. dev.')

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Chance', alpha=.8)
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])


In [ ]:
plt.xlabel('False Positive Rate',fontsize='20')
plt.ylabel('True Positive Rate',fontsize='xx-large')
plt.title('ROC 5-fold CV Comparison', fontsize='xx-large')
plt.legend(loc="lower right",fontsize=20)
plt.savefig('ROC5fold_All_Oversampled_inloop.png',dpi=300)
plt.show()
